# Ungraded Lab: Exploring Convolutions

In this lab, you will explore how convolutions work by creating a basic convolution on a 2D grayscale image. Start by importing everything you will need in this lab.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Ascent image that you will work with from the SciPy library
from scipy.datasets import ascent

First, you will load the [ascent](https://docs.scipy.org/doc/scipy/reference/generated/scipy.datasets.ascent.html) image from [SciPy](https://scipy.org/). It's a nice, built-in picture with lots of angles and lines. 

In [ ]:
# load the ascent image
ascent_image = ascent()

You can use the pyplot library to show the image so you'll know what it looks like.

In [ ]:
# Visualize the image
plt.grid(False)
plt.gray()
plt.axis('off')
plt.imshow(ascent_image)
plt.show()

The image is stored as a numpy array so you can create the transformed image by first copying that array. You can also get the dimensions of the image so you can loop over it later. 

In [ ]:
# Copy image to a numpy array
image_transformed = np.copy(ascent_image)

# Get the dimensions of the image
size_x = image_transformed.shape[0]
size_y = image_transformed.shape[1]

print(f"image_transformed has shape: {size_x, size_y}")

Now you can create a filter as a 3x3 array. 

In [ ]:
# Experiment with different values and see the effect
filter = [[0, 1, 0], [1, -4, 1], [0, 1, 0]]

# A couple more filters to try for fun!
# filter = [ [-1, -2, -1], [0, 0, 0], [1, 2, 1]]
# filter = [ [-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]

# If all the digits in the filter don't add up to 0 or 1, you 
# should probably do a weight to get it to do so
# so, for example, if your weights are 1,1,1 1,2,1 1,1,1
# They add up to 10, so you would set a weight of .1 if you want to normalize them
weight  = 1

Now you can create a convolution. You will iterate over the image, leaving a 1 pixel margin, and multiplying each of the neighbors of the current pixel by the value defined in the filter (i.e. the current pixel's neighbor above it and to the left will be multiplied by the top left item in the filter, etc.) 

You'll then multiply the result by the weight, and then ensure the result is in the range 0-255.

Finally you'll load the new value into the transformed image.

*Note: The main objective of this notebook is to simply demonstrate the effect of convolutions and pooling visually. Thus, you will skip some optimizations like dropping border pixels after convolutions, handling odd image dimensions in max pooling, etc.*

In [ ]:
# Iterate over the image
for x in range(1, size_x-1):
    for y in range(1, size_y-1):
        convolution = 0.0
        convolution = convolution + (ascent_image[x-1, y-1] * filter[0][0])
        convolution = convolution + (ascent_image[x-1, y] * filter[0][1])  
        convolution = convolution + (ascent_image[x-1, y+1] * filter[0][2])     
        convolution = convolution + (ascent_image[x, y-1] * filter[1][0])    
        convolution = convolution + (ascent_image[x, y] * filter[1][1])    
        convolution = convolution + (ascent_image[x, y+1] * filter[1][2])    
        convolution = convolution + (ascent_image[x+1, y-1] * filter[2][0])    
        convolution = convolution + (ascent_image[x+1, y] * filter[2][1])    
        convolution = convolution + (ascent_image[x+1, y+1] * filter[2][2])    
      
        # Multiply by weight
        convolution = convolution * weight   
      
        # Check the boundaries of the pixel values
        if(convolution < 0):
            convolution = 0
        if(convolution > 255):
            convolution = 255

        # Load into the transformed image
        image_transformed[x, y] = convolution

After the loop, you can now plot the image to see the effect of the convolution!

In [ ]:
# Plot the image. Note the size of the axes -- you will compare this later after max pooling
plt.gray()
plt.grid(False)
plt.imshow(image_transformed)
plt.show()   

## Effect of Max Pooling

The next cell will show a (2, 2) pooling. The idea here is to iterate over the image, and look at the pixel and it's immediate neighbors to the right, beneath, and right-beneath. It will take the largest of them and load it into the new image. Thus, the new image will be 1/4 the size of the old -- with the dimensions on X and Y being halved by this process. You'll see that the features get maintained despite this compression!

In [ ]:
# Assign dimensions half the size of the original image
new_x = int(size_x / 2)
new_y = int(size_y / 2)

# Create blank image with reduced dimensions
newImage = np.zeros((new_x, new_y))

# Iterate over the image
for x in range(0, size_x, 2):
    for y in range(0, size_y, 2):
    
        # Store all the pixel values in the (2,2) pool
        pixels = []
        pixels.append(image_transformed[x, y])
        pixels.append(image_transformed[x+1, y])
        pixels.append(image_transformed[x, y+1])
        pixels.append(image_transformed[x+1, y+1])

        # Get only the largest value and assign to the reduced image
        newImage[int(x / 2), int(y / 2)] = max(pixels)

# Plot the image. Note the size of the axes -- it is now half of the image loaded earlier
plt.gray()
plt.grid(False)
plt.imshow(newImage)
plt.show()